In [1]:
"""
Create database_file with common configuration of variable naming, columns based on different datasets.
For now we only use *.csv exports from EU database.
https://www.eea.europa.eu/en/datahub/datahubitem-view/fa8b1229-3db6-495d-b18e-9c9b3267c02b?activeAccordion=

This notebook needs to be executed for each dataset we want to use later in the data analysis and model training process.
"""

from datetime import datetime

import pandas as pd
from config import COLS_MAPPER, COLS_PRE_DROP, DENSITY_THRESHOLD, OUTPUT_DIR, RAW_DATA_FILES, raw_csv_to_proc_csv_converter

In [2]:
# read in raw files based on array in config (update index for different files)
# in real application case we'd consider implementation of a eu_preprocessing_pipeline e.g. through class representation
file = RAW_DATA_FILES[0]
df = pd.read_csv(file)

C:\Users\Mr_Di\AppData\Local\Temp\ipykernel_13420\2160831717.py:4: DtypeWarning: Columns (3,8,12,28,35,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30134963 entries, 0 to 30134962
Data columns (total 40 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Country               object 
 2   VFN                   object 
 3   Mp                    object 
 4   Mh                    object 
 5   Man                   object 
 6   MMS                   float64
 7   Tan                   object 
 8   T                     object 
 9   Va                    object 
 10  Ve                    object 
 11  Mk                    object 
 12  Cn                    object 
 13  Ct                    object 
 14  Cr                    object 
 15  r                     int64  
 16  m (kg)                float64
 17  Mt                    float64
 18  Enedc (g/km)          float64
 19  Ewltp (g/km)          float64
 20  W (mm)                float64
 21  At1 (mm)              float64
 22  At2 (mm)              float64
 23  Ft   

In [4]:
df.head(10)

,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,ech,RLFI,Electric range (km)
0,56002959,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-06-17,NaN,NaN,NaN,NaN
1,56002960,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-06-04,NaN,NaN,NaN,NaN
2,56002961,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-04-07,NaN,NaN,NaN,NaN
3,56002962,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-04-13,NaN,NaN,NaN,NaN
4,56002963,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-11-19,NaN,NaN,NaN,NaN
5,56002964,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-04-16,NaN,NaN,NaN,NaN
6,56002965,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-06-10,NaN,NaN,NaN,NaN
7,56002966,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-05-05,NaN,NaN,NaN,NaN
8,56002967,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-05-11,NaN,NaN,NaN,NaN
9,56002968,GR,IP-091932-KMH-1,HYUNDAI,HYUNDAI,HYUNDAI MOTOR COMPANY,NaN,e4*2007/46*1259*11,OS,F5D11,...,NaN,NaN,NaN,F,2021,2021-05-21,NaN,NaN,NaN,NaN


In [5]:
# Find fully identical duplicate rows
duplicates = df[df.duplicated(keep=False)]
print(duplicates)

Empty DataFrame
Columns: [ID, Country, VFN, Mp, Mh, Man, MMS, Tan, T, Va, Ve, Mk, Cn, Ct, Cr, r, m (kg), Mt, Enedc (g/km), Ewltp (g/km), W (mm), At1 (mm), At2 (mm), Ft, Fm, ec (cm3), ep (KW), z (Wh/km), IT, Ernedc (g/km), Erwltp (g/km), De, Vf, Status, year, Date of registration, Fuel consumption , ech, RLFI, Electric range (km)]
Index: []

[0 rows x 40 columns]


In [6]:
def analyse_col_duplicates(df, col="ID"):
    # function to analyse duplicates (optional usage)
    duplicate_counts = df[col].value_counts()
    print(duplicate_counts)
    duplicate_ids = duplicate_counts[duplicate_counts > 1].index
    df_with_duplicates_in_col = df[df[col].isin(duplicate_ids)]
    df_with_duplicates_in_col = df_with_duplicates_in_col.sort_values(by=col)
    # prepare output
    current_date = datetime.now().strftime('%Y_%m_%d')
    output_file = OUTPUT_DIR + current_date + "-duplicate_ids.csv"
    df_with_duplicates_in_col.to_csv(output_file, index=False)
    print("Exported csv for analysis to ", output_file)

def remove_duplicates_on_col_keep_none(df, col="ID"):
    # function to remove duplicates based on column
    # strategy: keep none of the duplicates
    print("Removing duplicates (keep none)")
    df = df[df.duplicated(col, keep=False) == False]
    # make sure we don't have any duplicates left
    assert df.duplicated(col).sum() == 0
    return df

In [7]:
# # x = x[~x.duplicated(keep=False)]
# x.duplicated(subset=['ID']).sum()
# x = x[x.duplicated('ID', keep=False) == False]
# print(x.duplicated(subset=['ID']).sum())
# x.set_index("ID", inplace=True)
# # show row with ID 2533
# x.loc[2533]

In [8]:
# set index by eu table id
index_duplicate_count = df.duplicated(subset=['ID']).sum()
duplicate_removal_threshold = 0.01 * len(df)
if index_duplicate_count != 0:
    # happened in last analysis of 30134963 rows (2023,22,21 eu data) for 273 rows
    print(f"Found {index_duplicate_count} duplicates")
    analyse_col_duplicates(df, col="ID")
    if index_duplicate_count < duplicate_removal_threshold:
        df = remove_duplicates_on_col_keep_none(df, col="ID")
    else:
        raise Exception("Found too many duplicates in the index column. Please check the data.")
df.set_index("ID", inplace=True)

Found 273 duplicates
ID
9            3
127          3
113          3
2457         2
2611         2
            ..
140000051    1
140000052    1
140000053    1
140000054    1
140000058    1
Name: count, Length: 30134690, dtype: int64
Exported csv for analysis to  files/output/2025_03_01-duplicate_ids.csv
Removing duplicates (keep none)


In [9]:
# Drop columns based on config COLS_PRE_DROP (learnings from data exploration)
df.drop(columns=COLS_PRE_DROP, inplace=True)

In [10]:
# Rename columns based on config COLS_MAPPER
df.rename(columns=COLS_MAPPER, inplace=True)
df.columns

Index(['member_state', 'manufacturer_name_eu', 'vehicle_type',
       'commercial_name', 'category_of_vehicle', 'mass_vehicle',
       'weltp_test_mass', 'specific_co2_emissions', 'fuel_type', 'fuel_mode',
       'engine_capacity', 'engine_power', 'electric_energy_consumption',
       'innovative_technologies', 'erwltp', 'year', 'fuel_consumption',
       'electric_range'],
      dtype='object')

In [11]:
# Give overview about missing percentages for each variable
missing_percentage = df.isna().sum() / len(df)
print(missing_percentage)

member_state                   0.000000
manufacturer_name_eu           0.000000
vehicle_type                   0.000429
commercial_name                0.003110
category_of_vehicle            0.001289
mass_vehicle                   0.000015
weltp_test_mass                0.012037
specific_co2_emissions         0.001166
fuel_type                      0.000000
fuel_mode                      0.000007
engine_capacity                0.131065
engine_power                   0.010614
electric_energy_consumption    0.789126
innovative_technologies        0.376596
erwltp                         0.383882
year                           0.000000
fuel_consumption               0.207794
electric_range                 0.799314
dtype: float64


In [12]:
# Delete based on threshold, but ignore cols which are part of COLS_MAPPER (indication of interesting vars)
# many variables/cols already dropped through COLS_PRE_DROP process
cols_to_be_dropped = list()
cols_drop_ignore = list()
for _, col in COLS_MAPPER.items():
    cols_drop_ignore.append(col)

for col, percentage in missing_percentage.items():
    if percentage > DENSITY_THRESHOLD and col not in cols_drop_ignore:
        cols_to_be_dropped.append(col)

print(f"Dropping: {missing_percentage[cols_to_be_dropped]}")

df.drop(columns=cols_to_be_dropped, inplace=True)

Dropping: Series([], dtype: float64)


In [13]:
# split variables into categorical and numerical
cat_vars = df.select_dtypes(include="object")
num_vars = df.select_dtypes(include=["float64", "int64"])

# reordering columns: categorical vars first, numerical vars next, targets at the end

potential_targets = ['electric_energy_consumption', 'fuel_consumption', 'specific_co2_emissions']
col_order = list(cat_vars.columns)  # Start with categorical variables
col_order.extend(num_vars.columns)  # Add numerical variables

# Move target variables to the end
for target in potential_targets:
    if target in col_order:
        col_order.remove(target)
    col_order.append(target)

# Apply the new column order to df1
df = df.loc[:, col_order]

In [14]:
# Give overview about changed dataset
df.head(5)

,member_state,manufacturer_name_eu,vehicle_type,commercial_name,category_of_vehicle,fuel_type,fuel_mode,innovative_technologies,mass_vehicle,weltp_test_mass,engine_capacity,engine_power,erwltp,year,electric_range,electric_energy_consumption,fuel_consumption,specific_co2_emissions
ID,,,,,,,,,,,,,,,,,,
56002959,GR,HYUNDAI,OS,"KONA,KAUAI",M1,DIESEL,M,NaN,1415.0,1600.0,1598.0,100.0,NaN,2021,NaN,NaN,NaN,127.0
56002960,GR,HYUNDAI,OS,"KONA,KAUAI",M1,DIESEL,M,NaN,1415.0,1600.0,1598.0,100.0,NaN,2021,NaN,NaN,NaN,127.0
56002961,GR,HYUNDAI,OS,"KONA,KAUAI",M1,DIESEL,M,NaN,1415.0,1600.0,1598.0,100.0,NaN,2021,NaN,NaN,NaN,127.0
56002962,GR,HYUNDAI,OS,"KONA,KAUAI",M1,DIESEL,M,NaN,1415.0,1600.0,1598.0,100.0,NaN,2021,NaN,NaN,NaN,127.0
56002963,GR,HYUNDAI,OS,"KONA,KAUAI",M1,DIESEL,M,NaN,1415.0,1600.0,1598.0,100.0,NaN,2021,NaN,NaN,NaN,127.0


In [15]:
# export Dataframe in .csv format
export_file_name = raw_csv_to_proc_csv_converter(file)
df.to_csv(export_file_name, index=True)

In [16]:
# Use this output to update config.py DATABASE_FILE_DTYPES
# replace dtype('O') with 'object'
# replace dtype('float64') with 'float64'
# replace dtype('int64') with 'int64'
df.dtypes.to_dict()

{'member_state': dtype('O'),
 'manufacturer_name_eu': dtype('O'),
 'vehicle_type': dtype('O'),
 'commercial_name': dtype('O'),
 'category_of_vehicle': dtype('O'),
 'fuel_type': dtype('O'),
 'fuel_mode': dtype('O'),
 'innovative_technologies': dtype('O'),
 'mass_vehicle': dtype('float64'),
 'weltp_test_mass': dtype('float64'),
 'engine_capacity': dtype('float64'),
 'engine_power': dtype('float64'),
 'erwltp': dtype('float64'),
 'year': dtype('int64'),
 'electric_range': dtype('float64'),
 'electric_energy_consumption': dtype('float64'),
 'fuel_consumption': dtype('float64'),
 'specific_co2_emissions': dtype('float64')}